In [1]:
import pandas as pd

In [2]:
# Load the street mapping data
zones_df = pd.read_csv("./Datasets/parking-zones-linked-to-street-segments.csv")

# Get unique street names that we care about (from your real-time data)
unique_streets = zones_df['OnStreet'].unique()
print(f"Found {len(unique_streets)} unique streets")

Found 166 unique streets


In [3]:
# Function to process each year's usage data
def process_year_data(year):
    """Process historical parking data for one year"""
    
    # Load the usage data for this year with correct file names
    if year == 2017:
        usage_df = pd.read_csv("./Datasets/On-street_Car_Parking_Sensor_Data_-_2017.csv")
    elif year == 2018:
        usage_df = pd.read_csv("./Datasets/On-street_Car_Parking_Sensor_Data_-_2018.csv")
    elif year == 2019:
        usage_df = pd.read_csv("./Datasets/On-street_Car_Parking_Sensor_Data_-_2019.csv")
    elif year == 2020:
        usage_df = pd.read_csv("./Datasets/On-street_Car_Parking_Sensor_Data_-_2020__Jan_-_May_.csv")
    
    # Clean up the data - handle different duration column names
    usage_df = usage_df.dropna(subset=['StreetName'])
    
    # Handle different duration column names by year
    if year == 2017:
        duration_col = 'DurationSeconds'
        time_multiplier = 1  # Already in seconds
    else:  # 2018-2020
        duration_col = 'DurationMinutes' 
        time_multiplier = 60  # Convert minutes to seconds
    
    # Check if duration column exists
    if duration_col not in usage_df.columns:
        print(f"Warning: {duration_col} not found in {year} data. Available columns: {usage_df.columns.tolist()}")
        return pd.DataFrame()  # Return empty dataframe
    
    # Convert duration to numeric and handle missing values
    usage_df[duration_col] = pd.to_numeric(usage_df[duration_col], errors='coerce')
    usage_df = usage_df.dropna(subset=[duration_col])
    
    # Convert all to seconds for consistency
    usage_df['DurationSeconds'] = usage_df[duration_col] * time_multiplier
    
    # Normalize street names to handle case differences
    # Convert zones data to uppercase for comparison
    valid_streets_upper = [street.upper() for street in zones_df['OnStreet'].unique()]
    
    # Convert historical street names to uppercase and filter
    usage_df['StreetName_Upper'] = usage_df['StreetName'].str.upper()
    usage_filtered = usage_df[usage_df['StreetName_Upper'].isin(valid_streets_upper)]
    
    # Sum duration by street (using normalized street name)
    street_usage = usage_filtered.groupby('StreetName_Upper').agg({
        'DurationSeconds': 'sum'
    }).reset_index()
    
    # Map back to original case from zones data
    street_name_mapping = dict(zip(
        [street.upper() for street in zones_df['OnStreet'].unique()],
        zones_df['OnStreet'].unique()
    ))
    
    street_usage['OnStreet'] = street_usage['StreetName_Upper'].map(street_name_mapping)
    street_usage = street_usage.drop('StreetName_Upper', axis=1)
    
    # Add year column
    street_usage['Year'] = year
    
    # Convert to hours for readability
    street_usage['DurationHours'] = street_usage['DurationSeconds'] / 3600
    
    return street_usage[['OnStreet', 'Year', 'DurationHours']]

In [4]:
# Process all 4 years (2017-2020)
all_years_data = []

for year in [2017, 2018, 2019, 2020]:
    try:
        year_data = process_year_data(year)
        all_years_data.append(year_data)
        print(f"Processed {year}: {len(year_data)} streets")
    except Exception as e:
        print(f"Error processing {year}: {e}")

Processed 2017: 93 streets


/var/folders/ww/_1pfw06d59s1mg8kl6rxm_3h0000gn/T/ipykernel_2710/1335899254.py:9: DtypeWarning: Columns (3,12) have mixed types. Specify dtype option on import or set low_memory=False.
  usage_df = pd.read_csv("./Datasets/On-street_Car_Parking_Sensor_Data_-_2018.csv")


Processed 2018: 97 streets


/var/folders/ww/_1pfw06d59s1mg8kl6rxm_3h0000gn/T/ipykernel_2710/1335899254.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  usage_df = pd.read_csv("./Datasets/On-street_Car_Parking_Sensor_Data_-_2019.csv")


Processed 2019: 97 streets


/var/folders/ww/_1pfw06d59s1mg8kl6rxm_3h0000gn/T/ipykernel_2710/1335899254.py:13: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  usage_df = pd.read_csv("./Datasets/On-street_Car_Parking_Sensor_Data_-_2020__Jan_-_May_.csv")


Processed 2020: 97 streets


In [5]:
# Combine all years
if all_years_data:
    historical_usage = pd.concat(all_years_data, ignore_index=True)
    
    # Create pivot table: Streets vs Years
    usage_summary = historical_usage.pivot(
        index='OnStreet', 
        columns='Year', 
        values='DurationHours'
    ).fillna(0)
    
    # Round to whole numbers
    usage_summary = usage_summary.round(0).astype(int)
    
    # Rename 2020 column to indicate it's partial data
    if 2020 in usage_summary.columns:
        usage_summary = usage_summary.rename(columns={2020: '2020 (Jan-May)'})
    
    # Remove the columns name label for cleaner output
    usage_summary.columns.name = None
    
    # Export to CSV
    usage_summary.to_csv('historical_street_usage_2017_2020.csv')
    
    print(f"\nHistorical analysis complete!")
    print(f"Exported data for {len(usage_summary)} streets")
    print(f"\nColumns in output: {list(usage_summary.columns)}")
    print(f"\nSample data (first 5 streets):")
    print(usage_summary.head())

else:
    print("No data processed successfully")


Historical analysis complete!
Exported data for 99 streets

Columns in output: [2017, 2018, 2019, '2020 (Jan-May)']

Sample data (first 5 streets):
                   2017    2018     2019  2020 (Jan-May)
OnStreet                                                
A'Beckett Street  17162  207986   181125           47136
Albert Street     28565  768962  1137324          266202
Aquitania Way      6266   73848    83826           29763
Augusta Avenue      601   32741    34381           13884
Aurora Lane        4102   96311    84107           29492
